# Multi-Agent Collaboration - DSL query use case

In [19]:
import logging
import boto3
import os
import json
import time
import zipfile
import subprocess
from textwrap import dedent

### Configure Logging

In [20]:
# -----------------------------------------------------------------------------
# Configure Logging
# -----------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

### Import Bedrock Agent utilities

In [21]:
from src.utils.bedrock_agent import Agent, SupervisorAgent, agents_helper, region, account_id

### Define AWS clients

In [22]:
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region_name=region)
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
iam_client = boto3.client('iam', region_name=region)
lambda_client = boto3.client('lambda', region_name=region)

logger.info(f"Region: {region}")
logger.info(f"Account ID: {account_id}")
logger.info(f"Agent Suffix: {agent_suffix}")


2025-02-07 18:55:31,802 - __main__ - INFO - Region: us-west-2
2025-02-07 18:55:31,802 - __main__ - INFO - Account ID: 533267284022
2025-02-07 18:55:31,803 - __main__ - INFO - Agent Suffix: us-west-2-533


### Helper Functions

In [23]:
def create_iam_role(role_name: str) -> str:
    """
    Creates or retrieves an IAM Role with the necessary trust policy for Lambda.
    Attaches AWSLambdaBasicExecutionRole, and adds inline policies for OpenSearch 
    and AOSS access.

    :param role_name: Name of the IAM Role to create or retrieve.
    :return: ARN of the created or retrieved IAM Role.
    """
    logger.info(f"Creating or retrieving IAM Role: {role_name}")
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    try:
        role = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
        )
        logger.info(f"IAM Role {role_name} created.")
    except iam_client.exceptions.EntityAlreadyExistsException:
        logger.info(f"IAM Role {role_name} already exists. Retrieving existing role.")
        role = iam_client.get_role(RoleName=role_name)

    # Attach AWS Lambda execution policy
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"
    )
    logger.info(f"Attached AWSLambdaBasicExecutionRole to {role_name}.")

    # Attach additional policies for OpenSearch access
    opensearch_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "es:Describe*",
                    "es:List*",
                    "es:Get*"
                ],
                "Resource": "*"
            }
        ]
    }
    opensearch_policy_name = f"{role_name}-OpenSearchPolicy"
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=opensearch_policy_name,
            PolicyDocument=json.dumps(opensearch_policy_document)
        )
        logger.info(f"Attached OpenSearch policy to IAM Role {role_name}.")
    except Exception as e:
        logger.error(f"Failed to attach OpenSearch policy to IAM Role {role_name}: {str(e)}")

    # Attach the new policy for aoss:APICall
    aoss_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "aoss:*"
                ],
                "Resource": "*"
            }
        ]
    }
    aoss_policy_name = f"{role_name}-AOSSPolicy"
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=aoss_policy_name,
            PolicyDocument=json.dumps(aoss_policy_document)
        )
        logger.info(f"Attached AOSS policy to IAM Role {role_name}.")
    except Exception as e:
        logger.error(f"Failed to attach AOSS policy to IAM Role {role_name}: {str(e)}")

    role_arn = role['Role']['Arn']

    # Wait for IAM role to propagate
    logger.info("Waiting 10 seconds for IAM role to propagate...")
    time.sleep(10)

    return role_arn


def create_lambda_package(source_file: str, zip_file_path: str, dependencies: list):
    """
    Packages a Lambda function and its dependencies into a single ZIP file.

    :param source_file: Path to the Lambda function source code.
    :param zip_file_path: Path to the ZIP file that will be created.
    :param dependencies: A list of Python packages required by the Lambda.
    """
    logger.info(f"Packaging Lambda function from {source_file} with dependencies {dependencies}")
    package_dir = "package"

    # Install dependencies to a local folder
    if not os.path.exists(package_dir):
        os.makedirs(package_dir)
    logger.info("Installing dependencies locally...")
    subprocess.run(
        f"pip install {' '.join(dependencies)} -t {package_dir}",
        shell=True,
        check=True
    )

    # Create ZIP file with dependencies and function
    logger.info(f"Creating Lambda deployment package: {zip_file_path}")
    with zipfile.ZipFile(zip_file_path, 'w') as zipf:
        # Add dependencies
        for root, _, files in os.walk(package_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, package_dir)
                zipf.write(file_path, arcname)

        # Add the Lambda function code
        zipf.write(source_file, os.path.basename(source_file))

    # Cleanup temporary package directory
    logger.info("Cleaning up temporary package directory...")
    subprocess.run(f"rm -rf {package_dir}", shell=True)
    logger.info("Lambda package created successfully.")


def create_lambda_function(function_name: str,
                           role_arn: str,
                           handler: str,
                           runtime: str,
                           zip_file_path: str,
                           region_name: str = region) -> dict:
    """
    Creates or updates an AWS Lambda function.

    :param function_name: Name of the Lambda function to create or update.
    :param role_arn: ARN of the IAM Role that Lambda will assume.
    :param handler: The function handler (e.g., 'index.lambda_handler').
    :param runtime: The Lambda runtime (e.g., 'python3.12').
    :param zip_file_path: Path to the ZIP file containing the Lambda code.
    :param region_name: AWS region where the Lambda will be created.
    :return: The response from the create_function or update_function_code API call.
    """
    logger.info(f"Creating/updating Lambda function: {function_name}")
    lambda_client = boto3.client('lambda', region_name=region_name)

    with open(zip_file_path, 'rb') as f:
        zip_content = f.read()

    try:
        response = lambda_client.create_function(
            FunctionName=function_name,
            Runtime=runtime,
            Role=role_arn,
            Handler=handler,
            Code={'ZipFile': zip_content},
            Description='Lambda function to execute DSL queries',
            Timeout=15,
            MemorySize=128,
            Publish=True
        )
        logger.info(f"Lambda function {function_name} created successfully.")
    except lambda_client.exceptions.ResourceConflictException:
        logger.info(f"Lambda function {function_name} already exists. Updating its code...")
        response = lambda_client.update_function_code(
            FunctionName=function_name,
            ZipFile=zip_content
        )
        logger.info(f"Lambda function {function_name} updated successfully.")

    return response


def add_resource_based_policy(function_name: str,
                              agent_ids: list,
                              region_name: str,
                              account_id: str):
    """
    Adds a resource-based policy to the specified Lambda function to allow invocation
    from one or more Bedrock agents.

    :param function_name: Name of the Lambda function.
    :param agent_ids: List of agent IDs permitted to invoke this Lambda.
    :param region_name: AWS region.
    :param account_id: AWS account ID.
    """
    logger.info(f"Adding resource-based policy to Lambda function {function_name} for agents: {agent_ids}")
    statement_id_prefix = "AllowExecutionFromBedrockAgent"
    policy_doc = {
        "Version": "2012-10-17",
        "Statement": []
    }

    for agent_id in agent_ids:
        sid = f"{statement_id_prefix}_{agent_id}"
        policy_doc['Statement'].append({
            "Sid": sid,
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "lambda:InvokeFunction",
            "Resource": f"arn:aws:lambda:{region_name}:{account_id}:function:{function_name}",
            "Condition": {
                "ArnLike": {
                    "AWS:SourceArn": f"arn:aws:bedrock:{region_name}:{account_id}:agent/{agent_id}"
                }
            }
        })

    # Retrieve existing policy and remove any existing statements with the same prefix
    try:
        existing_policy = lambda_client.get_policy(FunctionName=function_name)
        existing_policy_doc = json.loads(existing_policy['Policy'])
        for stmt in existing_policy_doc['Statement']:
            if stmt['Sid'].startswith(statement_id_prefix):
                sid_to_remove = stmt['Sid']
                logger.info(f"Removing existing statement: {sid_to_remove}")
                lambda_client.remove_permission(
                    FunctionName=function_name,
                    StatementId=sid_to_remove
                )
    except lambda_client.exceptions.ResourceNotFoundException:
        logger.info(f"No existing policy found for Lambda function {function_name}.")
    except Exception as e:
        logger.error(f"Error retrieving/removing existing policy for {function_name}: {str(e)}")

    # Add new permissions
    for stmt in policy_doc['Statement']:
        sid_val = stmt['Sid']
        try:
            lambda_client.add_permission(
                FunctionName=function_name,
                StatementId=sid_val,
                Action=stmt['Action'],
                Principal=stmt['Principal']['Service'],
                SourceArn=stmt['Condition']['ArnLike']['AWS:SourceArn']
            )
            logger.info(f"Added permission for statement: {sid_val}")
        except Exception as e:
            logger.error(f"Failed to add resource-based policy for {function_name}, statement {sid_val}: {str(e)}")


## Main Execution

In [24]:
# Load Shipping Schema 
with open('schemas/ecom_shipping_schema.json', 'r') as file:
    ecom_shipping_schema = json.load(file)
ecom_shipping_schema_string = json.dumps(ecom_shipping_schema, indent=2)

# Agent foundation model 
agent_foundation_model = [
    "anthropic.claude-3-5-sonnet-20241022-v2:0"
]

# Force re-create default setting for Agent objects, but for now set to False
Agent.set_force_recreate_default(True)

In [25]:
"""
Main execution flow:
    1. Create an IAM Role for Lambda.
    2. Create/Update two Lambda functions (execute-dsl-query, execute-modified-dsl-query).
    3. Create DSL Query Agent & Query Fixer Agent referencing those Lambda functions.
    4. Retrieve the newly created agent IDs.
    5. Add resource-based policies to each Lambda function for those agent IDs.
    6. Create the Supervisor Agent to orchestrate both DSL Query and Query Fixer agents.
    7. Invoke the Supervisor Agent with a sample query.
    8. Delete the agents (cleanup).
"""
# -------------------------------------------------------------------------
# 1. Create (or retrieve) IAM Role for Lambda
# -------------------------------------------------------------------------
IAM_ROLE_NAME = f"LambdaExecutionRole-{agent_suffix}"
role_arn = create_iam_role(IAM_ROLE_NAME)


# -------------------------------------------------------------------------
# 2. Create the first Lambda (execute-dsl-query)
# -------------------------------------------------------------------------
DSL_QUERY_LAMBDA_NAME = f"execute-dsl-query-{agent_suffix}"
DSL_QUERY_LAMBDA_PATH = "src/lambda/execute_dsl_query.py"
DSL_QUERY_ZIP_PATH = "dsl_query_function.zip"

if not os.path.exists(DSL_QUERY_LAMBDA_PATH):
    logger.error(f"Error: {DSL_QUERY_LAMBDA_PATH} does not exist.")

DEPENDENCIES = ["opensearch-py", "requests", "urllib3"]

# Package & create the Lambda
create_lambda_package(DSL_QUERY_LAMBDA_PATH, DSL_QUERY_ZIP_PATH, DEPENDENCIES)
create_lambda_function(
    function_name=DSL_QUERY_LAMBDA_NAME,
    role_arn=role_arn,
    handler="execute_dsl_query.lambda_handler",
    runtime="python3.12",
    zip_file_path=DSL_QUERY_ZIP_PATH
)
os.remove(DSL_QUERY_ZIP_PATH)


2025-02-07 18:55:31,829 - __main__ - INFO - Creating or retrieving IAM Role: LambdaExecutionRole-us-west-2-533
2025-02-07 18:55:32,206 - __main__ - INFO - IAM Role LambdaExecutionRole-us-west-2-533 already exists. Retrieving existing role.
2025-02-07 18:55:32,501 - __main__ - INFO - Attached AWSLambdaBasicExecutionRole to LambdaExecutionRole-us-west-2-533.
2025-02-07 18:55:32,656 - __main__ - INFO - Attached OpenSearch policy to IAM Role LambdaExecutionRole-us-west-2-533.
2025-02-07 18:55:32,810 - __main__ - INFO - Attached AOSS policy to IAM Role LambdaExecutionRole-us-west-2-533.
2025-02-07 18:55:32,811 - __main__ - INFO - Waiting 10 seconds for IAM role to propagate...
2025-02-07 18:55:42,819 - __main__ - INFO - Packaging Lambda function from src/lambda/execute_dsl_query.py with dependencies ['opensearch-py', 'requests', 'urllib3']
2025-02-07 18:55:42,821 - __main__ - INFO - Installing dependencies locally...


  Using cached opensearch_py-2.8.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached opensearch_py-2.8.0-py3-none-any.whl (353 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl (196 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cach

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.16.1 requires botocore<1.35.89,>=1.35.74, but you have botocore 1.36.9 which is incompatible.
datasets 2.21.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
awscli 1.34.8 requires botocore==1.35.8, but you have botocore 1.36.9 which is incompatible.
awscli 1.34.8 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.11.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
2025-02-07 18:55:43,919 - __main__ - INFO - Creating Lambda deployment package: dsl_query_function.zip
2025-02-07 18:55:43,958 - __main__ - INFO - Cleaning up temporary package directory...
2025-02-07 18

2025-02-07 18:55:46,299 - __main__ - INFO - Lambda function execute-dsl-query-us-west-2-533 already exists. Updating its code...
2025-02-07 18:55:47,891 - __main__ - INFO - Lambda function execute-dsl-query-us-west-2-533 updated successfully.


In [26]:
# -------------------------------------------------------------------------
# 2(b). Create the second Lambda (execute-modified-dsl-query)
# -------------------------------------------------------------------------
MODIFIED_QUERY_LAMBDA_NAME = f"execute-modified-dsl-query-{agent_suffix}"
MODIFIED_QUERY_LAMBDA_PATH = "src/lambda/execute_modified_dsl_query.py"
MODIFIED_QUERY_ZIP_PATH = "modified_query_function.zip"

if not os.path.exists(MODIFIED_QUERY_LAMBDA_PATH):
    logger.error(f"Error: {MODIFIED_QUERY_LAMBDA_PATH} does not exist.")

create_lambda_package(MODIFIED_QUERY_LAMBDA_PATH, MODIFIED_QUERY_ZIP_PATH, DEPENDENCIES)
create_lambda_function(
    function_name=MODIFIED_QUERY_LAMBDA_NAME,
    role_arn=role_arn,
    handler="execute_modified_dsl_query.lambda_handler",
    runtime="python3.12",
    zip_file_path=MODIFIED_QUERY_ZIP_PATH
)
os.remove(MODIFIED_QUERY_ZIP_PATH)

2025-02-07 18:55:47,907 - __main__ - INFO - Packaging Lambda function from src/lambda/execute_modified_dsl_query.py with dependencies ['opensearch-py', 'requests', 'urllib3']
2025-02-07 18:55:47,908 - __main__ - INFO - Installing dependencies locally...


  Using cached opensearch_py-2.8.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached opensearch_py-2.8.0-py3-none-any.whl (353 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl (196 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cach

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.16.1 requires botocore<1.35.89,>=1.35.74, but you have botocore 1.36.9 which is incompatible.
datasets 2.21.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
awscli 1.34.8 requires botocore==1.35.8, but you have botocore 1.36.9 which is incompatible.
awscli 1.34.8 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.11.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
2025-02-07 18:55:48,851 - __main__ - INFO - Creating Lambda deployment package: modified_query_function.zip
2025-02-07 18:55:48,889 - __main__ - INFO - Cleaning up temporary package directory...
2025-02-

2025-02-07 18:55:50,704 - __main__ - INFO - Lambda function execute-modified-dsl-query-us-west-2-533 already exists. Updating its code...
2025-02-07 18:55:52,299 - __main__ - INFO - Lambda function execute-modified-dsl-query-us-west-2-533 updated successfully.


In [27]:

# -------------------------------------------------------------------------
# 3. Create the DSL Query Agent & Query Fixer Agent
#
#    Important: reference the just-created Lambda ARNs in "tool_code"
#    The actual ARN is "arn:aws:lambda:<REGION>:<ACCOUNT>:function:<FUNCTION_NAME>"
# -------------------------------------------------------------------------
dsl_query_lambda_arn = f"arn:aws:lambda:{region}:{account_id}:function:{DSL_QUERY_LAMBDA_NAME}"
modified_query_lambda_arn = f"arn:aws:lambda:{region}:{account_id}:function:{MODIFIED_QUERY_LAMBDA_NAME}"

logger.info("Creating DSL Query Agent...")
dsl_query_agent = Agent.direct_create(
    name=f"dsl-query-agent-{agent_suffix}",
    role="DSL Query Creator",
    goal="Create DSL queries for a given user query",
    instructions=f"""
    You are an expert in generating Query DSL for Elasticsearch-style queries. Your task is to convert a 
    given natural language user question into a well-structured Query DSL.
    
    ## Instructions:
    - Use the provided e-commerce shipping schema to construct the query.
    - Encapsulate the output in <json>...</json> tags.
    - Follow the syntax of the Query DSL strictly; do not introduce elements outside the provided schema.
    
    ## Query Construction Rules:
    - **Keyword fields** (carrier, status, country): Use `term` for exact matches or `prefix`/`wildcard` for partial matches.
    - **Text fields** (description, address): Use `match` queries to account for analyzed terms.
    - **Nested fields** (tracking): Always use `nested` queries.
    - **Date fields**: Use `range` queries with date math for filtering by date ranges.
    - **Aggregations**: When counting occurrences, use a 'terms' aggregation on the relevant keyword field to capture the exact values present (e.g., 'delivery.carrier').
    - Break down complex queries into smaller parts for accuracy.
    - Think step-by-step before constructing the query.


    ## Schema:
    {ecom_shipping_schema_string}

    ## Output Format:
    - Return only the generated Query DSL within <json>...</json> tags.
    - Do not include explanations, comments, or additional text.
    """,
    tool_code=dsl_query_lambda_arn,
    tool_defs=[
        {
            "name": "execute_dsl_query",
            "description": "Executes a given DSL query and returns the results",
            "parameters": {
                "dsl_query": {
                    "description": "The DSL query to execute",
                    "type": "string",
                    "required": True,
                }
            }
        }
    ]
)

logger.info("Creating Query Fixer Agent...")
query_fixer_agent = Agent.direct_create(
    name=f"query-fixer-agent-{agent_suffix}",
    role="Query Repair Specialist",
    goal="Fix and optimize failed DSL queries",
    instructions=f"""
    You are an expert query debugger and optimizer. Your tasks are:
    1. Analyze failed DSL queries from the query generator
    2. Diagnose errors using OpenSearch error messages
    3. Apply targeted fixes while maintaining original intent
    4. Optimize queries for better recall when results are empty
    5. Extract exact terms from 'terms' aggregations for accurate reporting.
    6. Identify alternative ways to answer queries when direct fields are missing.
    7. Recognize schema gaps and propose workarounds or schema modifications.

    ## Repair Strategies:
    - SYNTAX ERRORS: Fix formatting issues in nested queries/aggregations
    - FIELD ERRORS: Map invalid fields to valid schema equivalents
    - ZERO HITS: Apply query relaxation techniques:
        * Add wildcards to keyword searches
        * Expand date ranges
        * Reduce strictness of term matches
        * Add synonym expansion
    - Ensure queries include 'terms' aggregations to capture exact values

    ## Optimization Rules:
    - Maintain original query structure where possible
    - Prefer query-time fixes over rearchitecting
    - Document all modifications in revision notes
    - Limit query relaxation to 3 iterations
    - When results are found, check the 'terms' aggregation for the exact field values.
    - Report the exact terms from the data (e.g., use "DHL Express" if that's the stored value).

    ## Schema Gap Analysis & Alternative Solutions:
    - Creative Field Mapping: If direct fields are missing, use existing fields to infer answers.
      * Example: If "delivery duration" is not available, compute it using `delivered_time - out_for_delivery_time`.
    - Schema Enhancement: Identify missing fields required for full query support.
    - Derived Data Solutions:
      * If exact data isn’t available but can be computed, create scripted fields using painless scripting.
      * If no alternative exists, clearly state the required data and suggest schema modifications.



    ## Schema:
    {ecom_shipping_schema_string}

    ## Output Format:
    - Return modified query in <json> tags
    - Include revision notes and exact terms from aggregations in <notes> tags 
    """,
    tool_code=modified_query_lambda_arn,
    tool_defs=[
        {
            "name": "retry_query",
            "description": "Retries a modified version of the failed query",
            "parameters": {
                "modified_dsl_query": {
                    "description": "The corrected DSL query",
                    "type": "string",
                    "required": True
                },
                "revision_notes": {
                    "description": "Description of modifications made",
                    "type": "string",
                },
            }
        }
    ]
)



# -------------------------------------------------------------------------
kb_rag_agent = Agent.direct_create(
    name=f"kb-response-agent-{agent_suffix}",
    role="Knowledge Base Content Analyzer",
    goal="Analyze retrieved document content and generate well-structured responses",
    instructions="""
    You analyze pre-retrieved document content and generate clear, accurate responses.

    ## Response Rules:
    - Synthesize information from provided passages
    - Include relevant quotes with proper citations
    - Use consistent citation format [doc_id:para_num]
    - Maintain factual accuracy
    - Flag any inconsistencies between sources

    ## Output Format:
    Response should be structured as:
    1. Direct answer
    2. Supporting evidence
    3. Source citations
    4. Confidence level (High/Medium/Low)

    ## Quality Guidelines:
    - Prefer direct quotes for key information
    - Summarize when appropriate
    - Note any information gaps
    - Maintain neutral tone
    """,
    kb_descr="Use knowledge base to extract relevant information, analyze content across multiple documents, and generate accurate responses with proper citations. Focus on maintaining context and factual accuracy.",
    kb_id="5GADU65GNF",
    verbose=True
)



2025-02-07 18:55:52,321 - __main__ - INFO - Creating DSL Query Agent...



Deleting existing agent and corresponding lambda for: dsl-query-agent-us-west-2-533...
Agent dsl-query-agent-us-west-2-533 not found
Creating agent dsl-query-agent-us-west-2-533...
Created agent, id: UZ0ZCHE67L, alias id: TSTALIASID

Adding action group with Lambda: arn:aws:lambda:us-west-2:533267284022:function:execute-dsl-query-us-west-2-533...
Waiting for agent status to change. Current status CREATING
Agent id UZ0ZCHE67L current status: NOT_PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id UZ0ZCHE67L current status: PREPARED


2025-02-07 18:56:20,595 - __main__ - INFO - Creating Query Fixer Agent...


DONE: Agent: dsl-query-agent-us-west-2-533, id: UZ0ZCHE67L, alias id: AGKDHUAHJJ


Deleting existing agent and corresponding lambda for: query-fixer-agent-us-west-2-533...
Agent query-fixer-agent-us-west-2-533 not found
Creating agent query-fixer-agent-us-west-2-533...
Created agent, id: BVZPDW8UGH, alias id: TSTALIASID

Adding action group with Lambda: arn:aws:lambda:us-west-2:533267284022:function:execute-modified-dsl-query-us-west-2-533...
Waiting for agent status to change. Current status CREATING
Agent id BVZPDW8UGH current status: NOT_PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id BVZPDW8UGH current status: PREPARED
DONE: Agent: query-fixer-agent-us-west-2-533, id: BVZPDW8UGH, alias id: KCBBPUCEVM


Deleting existing agent and corresponding lambda for: kb-response-agent-us-west-2-533...
Agent kb-response-agent-us-west-2-533 not found
Creating agent kb-response-agent-us-west-2-533...
Creating agent: kb-response-agent-us-west-2-533...
Created agent 

In [28]:
# -------------------------------------------------------------------------
# 4. Retrieve the newly created Agent IDs
# -------------------------------------------------------------------------
logger.info("Retrieving DSL Query Agent ID...")
dsl_query_agent_id = agents_helper.get_agent_id_by_name(dsl_query_agent.name)
logger.info(f"DSL Query Agent ID: {dsl_query_agent_id}")

logger.info("Retrieving Query Fixer Agent ID...")
query_fixer_agent_id = agents_helper.get_agent_id_by_name(query_fixer_agent.name)
logger.info(f"Query Fixer Agent ID: {query_fixer_agent_id}")

logger.info("Retrieving KB Response Agent ID...")
kb_rag_agent_id = agents_helper.get_agent_id_by_name(kb_rag_agent.name)
logger.info(f"KB Response Agent ID: {kb_rag_agent_id}")


2025-02-07 18:57:21,855 - __main__ - INFO - Retrieving DSL Query Agent ID...
2025-02-07 18:57:21,934 - __main__ - INFO - DSL Query Agent ID: UZ0ZCHE67L
2025-02-07 18:57:21,935 - __main__ - INFO - Retrieving Query Fixer Agent ID...
2025-02-07 18:57:22,026 - __main__ - INFO - Query Fixer Agent ID: BVZPDW8UGH
2025-02-07 18:57:22,027 - __main__ - INFO - Retrieving KB Response Agent ID...
2025-02-07 18:57:22,102 - __main__ - INFO - KB Response Agent ID: CRL3Q8ZW0K


In [29]:
# -------------------------------------------------------------------------
# 5. Add resource-based policy to each Lambda so the Agents can invoke them
# -------------------------------------------------------------------------
add_resource_based_policy(DSL_QUERY_LAMBDA_NAME, [dsl_query_agent_id], region, account_id)
add_resource_based_policy(MODIFIED_QUERY_LAMBDA_NAME, [query_fixer_agent_id], region, account_id)

2025-02-07 18:57:22,106 - __main__ - INFO - Adding resource-based policy to Lambda function execute-dsl-query-us-west-2-533 for agents: ['UZ0ZCHE67L']
2025-02-07 18:57:22,245 - __main__ - INFO - Removing existing statement: AllowExecutionFromBedrockAgent_YNAHU2DUTR
2025-02-07 18:57:22,429 - __main__ - INFO - Added permission for statement: AllowExecutionFromBedrockAgent_UZ0ZCHE67L
2025-02-07 18:57:22,429 - __main__ - INFO - Adding resource-based policy to Lambda function execute-modified-dsl-query-us-west-2-533 for agents: ['BVZPDW8UGH']
2025-02-07 18:57:22,478 - __main__ - INFO - Removing existing statement: AllowExecutionFromBedrockAgent_TN0JXYFRFN
2025-02-07 18:57:22,641 - __main__ - INFO - Added permission for statement: AllowExecutionFromBedrockAgent_BVZPDW8UGH


In [30]:
supervisor_agent = SupervisorAgent.direct_create(
    name=f"supervisor-agent-{agent_suffix}",
    role="Query Pipeline Orchestrator",
    collaboration_type="SUPERVISOR",
    collaborator_objects=[dsl_query_agent, query_fixer_agent, kb_rag_agent],
    collaborator_agents=[
        {
            "agent": dsl_query_agent.name,
            "instructions": dedent(f"""
                {dsl_query_agent.name} is responsible for generating the primary DSL query based on 
                the provided e-commerce shipping schema. Your task is to produce a precise Query DSL 
                encapsulated in <json>...</json> tags. Ensure the query strictly follows the schema 
                and DSL syntax without any additional commentary or explanations.
            """).strip(),
            "relay_conversation_history": "DISABLED"
        },
        {
            "agent": query_fixer_agent.name,
            "instructions": dedent(f"""
                Engage {query_fixer_agent.name} when any of the following conditions occur:
                1. The DSL query returns syntax or validation errors.
                2. The DSL query execution returns zero hits.
                3. The query requires optimization for improved recall.
                4. Alternative query solutions are needed due to schema limitations.
                
                Responsibilities:
                - Analyze error messages and the current query structure.
                - Apply targeted fixes that preserve the original query intent.
                - Implement query relaxation techniques (for example, adding wildcards, extending date ranges, or expanding term matches).
                - Identify and map alternative fields if direct schema fields are missing.
                - Suggest schema enhancements when appropriate.
                - Document all modifications with clear revision notes and output exact terms from aggregations.
                
                Return the corrected DSL query within <json>...</json> tags and include any revision notes within <notes>...</notes> tags.
            """),
            "relay_conversation_history": "DISABLED"
        },
        {
            "agent": kb_rag_agent.name,
            "instructions": dedent(f"""
                Engage {kb_rag_agent.name} to answer user questions that require analyzing the document content retrieved from executed queries.
                When search results are available, your task is to:
                
                1. Synthesize and validate the information from the provided passages.
                2. Generate a final response that includes a direct answer and supporting evidence with relevant quotes and citations.
                
                Your output must be clear, well-structured, and factually accurate to support decision-making.
            """),
            "relay_conversation_history": "DISABLED"
        }
    ],
    instructions=dedent(f"""
        High-Level Overview:
        Route user queries to the appropriate agent based on the type of answer required:
          - Structured Data Retrieval: If the query requires retrieving structured information from the e-commerce shipping data, it is routed to {dsl_query_agent.name}. If the DSL query returns errors or zero hits, the query is immediately routed to the Query Fixer Agent for reattempts.
          - Document Content Analysis: If the query requires a synthesized, in-depth answer derived from analyzing executed query results, it is routed to {kb_rag_agent.name}.
        
        Detailed Instructions:
        
        Route A: Structured Data Retrieval (DSL Query Agent + Query Fixer Agent)
        1. Initial Query Analysis:
           - Receive the user's natural language query.
           - Determine if the query requires structured data retrieval from the e-commerce shipping data.
           - Validate the query against the provided schema:
             {ecom_shipping_schema_string}
           - If the query qualifies, route it to {dsl_query_agent.name}.
        
        2. DSL Query Execution:
           - {dsl_query_agent.name} generates a Query DSL that is encapsulated in <json>...</json> tags and follows the provided schema.
        
        3. Error Handling & Retry:
           - Monitor the query execution results:
             a. If the DSL query returns syntax or validation errors, or if the result is zero hits, capture the error context.
             b. Immediately route the query, along with diagnostic details, to {query_fixer_agent.name}.
             c. {query_fixer_agent.name} applies targeted fixes and query relaxation techniques, then returns a modified DSL query.
             d. Validate the modified query; allow up to 3 retry attempts if necessary.
        
        4. Evaluation & Final Approval (for structured data queries):
           - Confirm that the final DSL query adheres to best practices (for example, proper nested queries, correct field types and mappings).
           - Maintain an audit trail of all query versions and modifications.
           - Generate an execution summary including:
             - Query versions attempted.
             - Reasons for modifications.
             - Performance metrics.
        
        General Aggregation Guidance:
           - If an aggregation returns an unexpectedly inflated count, verify whether the aggregation is counting nested or repeated values.
           - To accurately count unique items, use a cardinality aggregation on a unique field identifier rather than aggregating on fields that might contain duplicate entries.
        
        Route B: Document Content Analysis (KB Response Agent)
        1. Initial Query Analysis:
           - Receive the user's natural language query.
           - Determine if the query requires synthesizing and analyzing document content from executed queries.
           - If so, route the query to {kb_rag_agent.name}.
        
        2. KB Response Generation:
           - {kb_rag_agent.name} synthesizes and validates the information from the provided passages.
           - Generate a final response that includes:
             a. A direct answer.
             b. Supporting evidence with relevant quotes and citations.
             c. A confidence level.
        
        3. Final Response:
           - Deliver the final, well-structured answer to the user by addressing the user by name. 
    """)
)


Agent supervisor-agent-us-west-2-533 not found

Created supervisor, id: PTF2OIBXNT, alias id: TSTALIASID

  associating sub-agents / collaborators to supervisor...
Waiting for agent status to change. Current status CREATING
Agent id PTF2OIBXNT current status: NOT_PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id PTF2OIBXNT current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id PTF2OIBXNT current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id PTF2OIBXNT current status: PREPARED
DONE: Agent: supervisor-agent-us-west-2-533, id: PTF2OIBXNT, alias id: SM0GTIQCW6



In [31]:
# Inspect attributes of the SupervisorAgent object
# print(dir(supervisor_agent))


In [32]:
# Retrieve the Supervisor Agent ID
logger.info("Retrieving Supervisor Agent ID...")
supervisor_agent_id = agents_helper.get_agent_id_by_name(supervisor_agent.name)
logger.info(f"Supervisor Agent ID: {supervisor_agent_id}")

2025-02-07 18:57:58,514 - __main__ - INFO - Retrieving Supervisor Agent ID...
2025-02-07 18:57:58,588 - __main__ - INFO - Supervisor Agent ID: PTF2OIBXNT


In [ ]:
# Lets get the agent based on the ID
# supervisor_agentV2 = agents_helper.get_agent_by_id("KGTOVCVLKI")

In [33]:
# -------------------------------------------------------------------------
# 7. Invoke the Supervisor Agent with a sample query
# -------------------------------------------------------------------------
response = supervisor_agent.invoke(
    input_text="How many orders have been shipped by DHL?",
    session_id="12345",
    enable_trace=True,
    trace_level="core"
)
logger.info(f"Supervisor agent response: {response}")

invokeAgent API request ID: dde28364-d966-4c2a-a6f7-7b0dcc910abb
invokeAgent API session ID: 12345
  agent id: PTF2OIBXNT, agent alias id: SM0GTIQCW6
---- Step 1 ----
Took 4.6s, using 2797 tokens (in: 2651, out: 146) to complete prior action, observe, orchestrate.
This query requires structured data retrieval to count orders shipped by DHL carrier. I'll route this to the DSL query agent to create an appropriate query.
Using sub-agent collaborator: 'dsl-query-agent-us-west-2-533 [UZ0ZCHE67L/AGKDHUAHJJ]' passing input text:
Create a DSL query to count the number of orders where the delivery carrier is DHL.

---- Step 1.1 [using sub-agent name:dsl-query-agent-us-west-2-533, id:UZ0ZCHE67L/AGKDHUAHJJ] ----
Took 8.5s, using 1880 tokens (in: 1488, out: 392) to complete prior action, observe, orchestrate.
To create a DSL query that counts the number of orders where the delivery carrier is DHL, we need to use a term query to match the exact carrier name and then use a count aggregation. Let's c

2025-02-07 18:59:24,927 - __main__ - INFO - Supervisor agent response: There is 1 order that has been shipped by DHL (specifically DHL Express).


---- Step 1.9 [using sub-agent name:query-fixer-agent-us-west-2-533, id:BVZPDW8UGH/KCBBPUCEVM] ----
Took 3.3s, using 3344 tokens (in: 3233, out: 111) to complete prior action, observe, orchestrate.
Now I can provide the accurate answer to the user.
Final response:
There is 1 order that has been shipped by DHL (specifically DHL Express)....
Agent made a total of 10 LLM calls, using 28738 tokens (in: 26225, out: 2513), and took 67.5 total seconds


In [ ]:
# -------------------------------------------------------------------------
# 7. Invoke the Supervisor Agent with a sample query
# -------------------------------------------------------------------------
response = supervisor_agent.invoke(
    input_text="How many orders have been shipped by DHL?",
    session_id="12346",
    enable_trace=True,
    trace_level="core",
    session_state={
        "sessionAttributes": {
            "userName": "Alice"
        }
    }

)
logger.info(f"Supervisor agent response: {response}")

invokeAgent API request ID: 350bf5c5-5bed-4b3f-81af-fc0798c1caf7
invokeAgent API session ID: 12345
  agent id: D059VOSRUS, agent alias id: XBA6NRSJJ4
---- Step 1 ----
Took 4.3s, using 3505 tokens (in: 3341, out: 164) to complete prior action, observe, orchestrate.
I need to create a query to count orders where the delivery carrier is DHL. I'll send this request to the DSL query agent to generate the appropriate query.
Using sub-agent collaborator: 'dsl-query-agent-us-west-2-533 [YNAHU2DUTR/WJXPCPQSDM]' passing input text:
Create a query to count the number of orders where the delivery carrier contains 'DHL' (using a wildcard or match query to catch variations like 'DHL Express').

---- Step 1.1 [using sub-agent name:dsl-query-agent-us-west-2-533, id:YNAHU2DUTR/WJXPCPQSDM] ----
Took 7.6s, using 2512 tokens (in: 2081, out: 431) to complete prior action, observe, orchestrate.
To create a query that counts the number of orders where the delivery carrier contains 'DHL', we need to use a mor

2025-02-07 18:45:08,167 - __main__ - INFO - Supervisor agent response: There is 1 order that has been shipped by DHL.


---- Step 1.3 [using sub-agent name:dsl-query-agent-us-west-2-533, id:YNAHU2DUTR/WJXPCPQSDM] ----
Took 2.8s, using 3652 tokens (in: 3549, out: 103) to complete prior action, observe, orchestrate.
Let me send this information to the user.
Final response:
There is 1 order that has been shipped by DHL....
Agent made a total of 4 LLM calls, using 12362 tokens (in: 11597, out: 765), and took 17.7 total seconds


invokeAgent API input parameters: input_text: How many orders have been shipped by DHL?, agent_id: QDMOLJO9TV, agent_alias_id: LXJCIDHVKO, session_id: 12345, session_state: {}, enable_trace: True, end_session: False, trace_level: all, multi_agent_names: {'CSKABIRKLE/V6WM6PSNHT': 'dsl-query-agent-us-west-2-533', 'SGSWZUVNOB/7KNLAUFPQC': 'query-fixer-agent-us-west-2-533', 'PM73T3Y9BR/GHH3JVSG1F': 'kb-response-agent-us-west-2-533', 'QDMOLJO9TV/LXJCIDHVKO': 'supervisor-agent-us-west-2-533'}


In [ ]:
# if isinstance(response, dict):
#     print(f"Answer: {response['answer']}")
#     print(f"Duration: {response['metadata']['duration_seconds']}s")
#     print(f"Total tokens: {response['metadata']['total_tokens']}")
#     # Save trace data if needed
#     with open('trace.json', 'w') as f:
#         json.dump(response['trace'], f, indent=2)
# else:
#     print(response)  # Just print the answer for other trace_level values

In [19]:
response = supervisor_agent.invoke(
    input_text="What are the effects of Covid-19 on e-commerce?",
    session_id="1245",
    enable_trace=True,
    trace_level="core"
)

invokeAgent API input parameters: input_text: What are the effects of Covid-19 on e-commerce?, agent_id: QDMOLJO9TV, agent_alias_id: LXJCIDHVKO, session_id: 1245, session_state: {}, enable_trace: True, end_session: False, trace_level: core, multi_agent_names: {'CSKABIRKLE/V6WM6PSNHT': 'dsl-query-agent-us-west-2-533', 'SGSWZUVNOB/7KNLAUFPQC': 'query-fixer-agent-us-west-2-533', 'PM73T3Y9BR/GHH3JVSG1F': 'kb-response-agent-us-west-2-533', 'QDMOLJO9TV/LXJCIDHVKO': 'supervisor-agent-us-west-2-533'}
Took 8.0s, using 2897 tokens.
This question requires analyzing shipping data patterns and trends related to Covid-19. I should route this to both the DSL query agent to get relevant data and then to the KB response agent for a comprehensive analysis.

First, I'll request a query to analyze shipping patterns during the Covid-19 period (focusing on 2020-2021) and compare it with pre-Covid data.
Took 11.0s, using 2118 tokens.
To analyze shipping patterns during the Covid-19 period (2020-2021) compare

In [ ]:
# response = supervisor_agent.invoke(
#     input_text="How many orders have recipients in Spain and were last updated during customs clearance after January 16, 2024?",
#     session_id="1245",
#     enable_trace=True,
#     trace_level="core"
# )

In [ ]:
# response = supervisor_agent.invoke(
#     input_text="What are the temperature controlled packages delivered within 2 hours?",
#     session_id="1245",
#     enable_trace=True,
#     trace_level="core"
# )

In [ ]:
# # -------------------------------------------------------------------------
# # 8. Cleanup: Delete the created agents
# # -------------------------------------------------------------------------
# logger.info("Deleting Supervisor Agent...")
# agents_helper.delete_agent(supervisor_agent.name, verbose=True)

# logger.info("Deleting DSL Query Agent...")
# agents_helper.delete_agent(dsl_query_agent.name, verbose=True)

# logger.info("Deleting Query Fixer Agent...")
# agents_helper.delete_agent(query_fixer_agent.name, verbose=True)

# logger.info("Deleting KB Response Agent...")
# agents_helper.delete_agent(kb_rag_agent.name, verbose=True)

2025-02-07 18:59:55,424 - __main__ - INFO - Deleting Supervisor Agent...


Found target agent, name: supervisor-agent-us-west-2-533, id: PTF2OIBXNT
Deleting aliases for agent PTF2OIBXNT...
Deleting alias SM0GTIQCW6 from agent PTF2OIBXNT
Deleting alias TSTALIASID from agent PTF2OIBXNT
Deleting agent: PTF2OIBXNT...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_supervisor-agent-us-west-2-533...


2025-02-07 19:00:07,254 - __main__ - INFO - Deleting DSL Query Agent...


Found target agent, name: dsl-query-agent-us-west-2-533, id: UZ0ZCHE67L
Deleting aliases for agent UZ0ZCHE67L...
Deleting alias AGKDHUAHJJ from agent UZ0ZCHE67L
Deleting alias TSTALIASID from agent UZ0ZCHE67L
Deleting agent: UZ0ZCHE67L...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_dsl-query-agent-us-west-2-533...


2025-02-07 19:00:19,581 - __main__ - INFO - Deleting Query Fixer Agent...


Found target agent, name: query-fixer-agent-us-west-2-533, id: BVZPDW8UGH
Deleting aliases for agent BVZPDW8UGH...
Deleting alias KCBBPUCEVM from agent BVZPDW8UGH
Deleting alias TSTALIASID from agent BVZPDW8UGH
Deleting agent: BVZPDW8UGH...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_query-fixer-agent-us-west-2-533...


2025-02-07 19:00:31,450 - __main__ - INFO - Deleting KB Response Agent...


Found target agent, name: kb-response-agent-us-west-2-533, id: CRL3Q8ZW0K
Deleting aliases for agent CRL3Q8ZW0K...
Deleting alias J6KX4IIFJQ from agent CRL3Q8ZW0K
Deleting alias TSTALIASID from agent CRL3Q8ZW0K
Deleting agent: CRL3Q8ZW0K...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_kb-response-agent-us-west-2-533...
